<a href="https://colab.research.google.com/github/msc-acse/acse-9-independent-research-project-Garethlomax/blob/offline_local/mastersproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# test implementation of lstm, convlstm and cnn lstm in pytorch

# IMPORT - TORCH AND MOVING MNIST.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !ls

%cd /content/drive/My \Drive/masters_project/python_modules/MovingMNIST-master

# all torch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torchvision.transforms as transforms
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit

import matplotlib.pyplot as plt
import h5py

# importing moving mnist test set.
from MovingMNIST import MovingMNIST

%cd /content/drive/My \Drive/masters_project/python_modules/pytorch-summary-master
from torchsummary import summary

# %cd /content/drive/My \Drive/masters_project/python_modules/pytorch_modelsize-master

%cd /content/drive/My \Drive/masters_project/python_modules/pytorchvis-master

!pip install torchviz

torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True



/content/drive/My Drive/masters_project/python_modules/MovingMNIST-master
/content/drive/My Drive/masters_project/python_modules/pytorch-summary-master
[Errno 2] No such file or directory: '/content/drive/My Drive/masters_project/python_modules/pytorchvis-master'
/content/drive/My Drive/masters_project/python_modules/pytorch-summary-master


In [3]:
h5py.run_tests()

.....................................................x...................................................................x....................................s...s......ss.......................................................................................................ssssss...................................................................x....x.........................x......x.................................................ssss..................
----------------------------------------------------------------------
Ran 457 tests in 2.152s

OK (skipped=14, expected failures=6)


<unittest.runner.TextTestResult run=457 errors=0 failures=0>

# Snippet to investigate gpu ram 


CITE THIS 

In [4]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 12.8 GB  | Proc size: 311.4 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


# OTHER IMPORTS'

In [0]:
from torchvision import models
alexnet = models.AlexNet()

In [0]:
# %pwd
# %cd ../MovingMNIST-master/
# from MovingMNIST import MovingMNIST


In [0]:
# printm()
# train_set = MovingMNIST(root='.data/mnist', train=True, download=False)
# test_set = MovingMNIST(root='.data/mnist', train=False, download=False)
# printm()

In [0]:
# test_set

In [0]:
# x = torch.randn((1,3,256,256))
# alexnet(x)


# CUDA CODE. 

In [8]:

device = 'cpu'
if torch.cuda.device_count() > 0 and torch.cuda.is_available():
    print("Cuda installed! Running on GPU!")
    device = 'cuda'
else:
    print("No GPU available!")
    
    
import random
def set_seed(seed):
    """
    Use this to set ALL the random seeds to a fixed value and take out any randomness from cuda kernels
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.benchmark = True  ##uses the inbuilt cudnn auto-tuner to find the fastest convolution algorithms. -
    torch.backends.cudnn.enabled   = True

    return True
  
set_seed(42)

Cuda installed! Running on GPU!


True

# MOVING MNIST

In [0]:
# train_set = MovingMNIST(root='.data/mnist', train=True, download=True)


In [0]:
# len(train_set)
# of dimensions, sample, test data / predictordata, frame
#train_set[0][2].shape
# size = train_set[8999][0].element_size() * train_set[8999][0].nelement() #print(x.element_size() * x.nelement()/ 1000000)
# print(size * 9000 * 2/ 1000000)



In [0]:
# examining video sequences.
# for i in range(10):
#     plt.figure()
#     plt.imshow(train_set[0][0][i].numpy())

# for i in range(10):
#     plt.figure()
#     plt.imshow(train_set[0][1][i].numpy())




# LSTM CELL AND MODEL

Based on lstm model from weather paper and others. 

pseudo code for lstm: 

http://people.idsia.ch/~juergen/lstm/sld024.htm





In [0]:
# now we start lstm cell

"""TODO: CUDIFY EVERYTHING"""





class LSTMunit(nn.Module):
    def __init__(self, input_channel_no, hidden_channels_no, kernel_size, stride = 1):
        super(LSTMunit, self).__init__()
        """base unit for an overall convLSTM structure. convLSTM exists in keras but
        not pytorch. LSTMunit repersents one cell in an overall convLSTM encoder decoder format
        the structure of convLSTMs lend themselves well to compartmentalising the LSTM
        cells. 
    
        Each cell takes an input the data at the current timestep Xt, and a hidden
        representation from the previous timestep Ht-1
    
        Each cell outputs Ht
        """
    
    
        self.input_channels = input_channel_no
    
        self.output_channels = hidden_channels_no
    
        self.kernel_size = kernel_size
    
        self.padding = (int((self.kernel_size - 1) / 2 ), int((self.kernel_size - 1) / 2 ))#to ensure output image same dims as input
        # as in conv nowcasting - see references 
        self.stride = stride # for same reasons as above
        
        # need convolutions, cells, tanh, sigmoid?
        # need input size for the lstm - on size of layers.
        # cannot do this because of the modules not being registered when stored in a list
        # can if we convert it to a parameter dict
    
        # list of names of filter to put in dictionary.
        # some of these are not convolutions
        """TODO: CHANGE THIS LAYOUT OF CONVOLUTIONAL LAYERS. """
        
        
        
        self.filter_name_list = ['Wxi', 'Wxf', 'Wxc', 'Wxo','Whi', 'Whf', 'Whc', 'Who']
        
        """ TODO : DEAL WITH BIAS HERE. """ 
        """ TODO: CAN INCLUDE BIAS IN ONE OF THE CONVOLUTIONS BUT NOT ALL OF THEM - OR COULD INCLUDE IN ALL? """

        # list of concolution instances for each lstm cell step
       #  nn.Conv2d(1, 48, kernel_size=3, stride=1, padding=0),
        self.conv_list = [nn.Conv2d(self.input_channels, self.output_channels, kernel_size =  self.kernel_size, stride = self.stride, padding = self.padding, bias = False).cuda() for i in range(4)]
        self.conv_list = self.conv_list + [(nn.Conv2d(self.output_channels, self.output_channels, kernel_size =  self.kernel_size, stride = self.stride, padding = self.padding, bias = True).cuda()).double() for i in range(4)]
#         self.conv_list = nn.ModuleList(self.conv_list)
        # stores nicely in dictionary for compact readability.
        # most ML code is uncommented and utterly unreadable. Here we try to avoid this
        self.conv_dict = nn.ModuleDict(zip(self.filter_name_list, self.conv_list))
    
        # may be able to combine all the filters and combine all the things to be convolved - as long as there is no cross layer convolution
        # technically the filter will be the same? - check this later.
    
        # set up W_co, W_cf, W_co as variables.
        """ TODO: decide whether this should be put into function. """
        
        
        """TODO: put correct dimensions of tensor in shape"""
        
        # of dimensions seq length, hidden layers, height, width
        """TODO: DEFINE THESE SYMBOLS. """
        """TODO: PUT THIS IN CONSTRUCTOR."""
        shape = [1, self.output_channels, 64, 64]
        
        self.Wco = nn.Parameter((torch.zeros(shape).double()).cuda(), requires_grad = True)
        self.Wcf = nn.Parameter((torch.zeros(shape).double()).cuda(), requires_grad = True)
        self.Wci = nn.Parameter((torch.zeros(shape).double()).cuda(), requires_grad = True)
#         self.Wco.name = "test"
#         self.Wco = torch.zeros(shape, requires_grad = True).double()
#         self.Wcf = torch.zeros(shape, requires_grad = True).double()
#         self.Wci = torch.zeros(shape, requires_grad = True).double()

        # activation functions.
        self.tanh = torch.tanh
        self.sig  = torch.sigmoid

#     (1, 6, kernel_size=5, padding=2, stride=1).double()
    def forward(self, x, h, c):
        """ put the various nets in here - instanciate the other convolutions."""
        """TODO: SORT BIAS OUT HERE"""
        """TODO: PUT THIS IN SELECTOR FUNCTION? SO ONLY PUT IN WXI ECT TO MAKE EASIER TO DEBUG?"""
#         print("size of x is:")
#         print(x.shape)
        # ERROR IS IN LINE 20
        #print(self.conv_dict['Wxi'](x).shape)
#         print("X:")
#         print(x.is_cuda)
#         print("H:")
#         print(h.is_cuda)
#         print("C")
#         print(c.is_cuda)
        
        i_t = self.sig(self.conv_dict['Wxi'](x) + self.conv_dict['Whi'](h) + self.Wci * c)
        f_t = self.sig(self.conv_dict['Wxf'](x) + self.conv_dict['Whf'](h) + self.Wcf * c)
        c_t = f_t * c + i_t * self.tanh(self.conv_dict['Wxc'](x) + self.conv_dict['Whc'](h))
        o_t = self.sig(self.conv_dict['Wxo'](x) + self.conv_dict['Who'](h) + self.Wco * c_t)
        h_t = o_t * self.tanh(c_t)
        
        return h_t, c_t
    
    def copy_in(self):
        """dummy function to copy in the internals of the output in the various architectures i.e encoder decoder format"""

In [10]:
9000/ 20 * 0.6


270.0

In [11]:
printm()

Gen RAM Free: 12.5 GB  | Proc size: 560.3 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [0]:
# test1 = (LSTMunit(20,20,5).double()).cuda()
# test1 = (LSTMunit(20,20,5).float()).cuda()
# # x = torch.randn((20,30,20,32,32)).double()


# x = torch.randn((20, 20, 32,32)).double()
# x = x.cuda()
# print(x.element_size() * x.nelement()/ 1000000)
# import time
# start = time.time()


# for i in range(20):
#     x, _ = test1(x,x,x)
# "the code you want to test stays here"
# end = time.time()
# print(end - start)
# ans, _ = test1(x,x,x)
# print(ans.shape)
# shape = [1,1,8,8]
# summary(test1, [(1,224,224),(3,224,224),(3,224,224)])

# from torchvision import models
# vgg = models.vgg16().to(device)

# summary(vgg, (3, 224, 224))

# alexnet = models.AlexNet().to(device)
# summary(alexnet, (3,224,224))

In [0]:
# from pytorch_modelsize import SizeEstimator
# se = SizeEstimator(test1, input_size=(20, 20, 32,32))
# print(se.estimate_size())



In [0]:
# for param in test2.parameters():
#     print(type(param.data), param.size(), param.is_cuda, param.name)

In [0]:
# #input_channel_no, hidden_channels_no, kernel_size, stride = 1):
# shape = [20,1,32,32]
# """TODO: IMPORTANT: STRIDE MUST BE KEPT AT 1 TO NOT DEPRECIATE THE SHAPE OF THE INPUT."""
# x = torch.randn(shape)
# h = torch.randn(shape)
# c= torch.randn(shape)

# # STRIDE ISNT WORKING - IS ONLY 1. 
# test = LSTMunit(1, 3, 5, 1)
# hout, cout = test(x,h,c)


# hout.shape

In [0]:
# hout.shape



In [15]:
isinstance([], list)

True

# LSTM FULL UNIT



In [0]:
"""TODO: IMPORTANT 
WHEN COPYING STATES OVER, INITIAL STATE OF DECODER IS BOTH LAST H AND LAST C 
FROM THE LSTM BEING COPIED FROM.

WE ALSO NEED TO INCLUDE THE ABILITY TO OUTPUT THE LAST H AND C AT EACH TIMESTEP
AS INPUT.
"""


""" SEQUENCE, BATCH SIZE, LAYERS, HEIGHT, WIDTH"""

class LSTMmain(nn.Module):
    
    
    """ collection of units to form encoder/ decoder branches - decide which are which
    need funcitonality to copy in and copy out outputs.
    
    
    layer output is array of booleans selectively outputing for each layer i.e 
    for three layer can have output on second and third but not first with 
    layer_output = [0,1,1]"""
    
    """TODO: DECIDE ON OUTPUT OF HIDDEN CHANNEL LIST """
    def __init__(self, shape, input_channel_no, hidden_channel_no, kernel_size, layer_output, test_input, debug = False, save_outputs = True, decoder = False):
        super(LSTMmain, self).__init__()
        
        """TODO: USE THIS AS BASIS FOR ENCODER DECODER."""
        """TODO: SPECIFY SHAPE OF INPUT VECTOR"""
        
        """TODO: FIGURE OUT HOW TO IMPLEMENT ENCODER DECODER ARCHITECUTRE"""
        self.test_input = test_input
        
        self.debug = debug
        
        self.save_all_outputs = save_outputs
        
        self.shape = shape
        
        """specify dimensions of shape - as in channel length ect. figure out once put it in a dataloader"""
        
        self.layers = len(test_input) #number of layers in the encoder. 
        
        self.seq_length = shape[1]
        
        self.enc_len = len(shape)
        
        self.input_chans = input_channel_no
        
        self.hidden_chans = hidden_channel_no
        
        self.kernel_size = kernel_size
        
        self.layer_output = layer_output
        
        # initialise the different conv cells. 
#         self.unit_list = [LSTMunit(input_channel_no, hidden_channel_no, kernel_size) for i in range(self.enc_len)]
        self.dummy_list = [input_channel_no] + list(self.test_input) # allows test input to be an array
        if self.debug:
            print("dummy_list:")
            print(self.dummy_list)
            
#         self.unit_list = nn.ModuleList([(LSTMunit(self.dummy_list[i], self.dummy_list[i+1], kernel_size).double()).cuda() for i in range(len(self.test_input))])
        self.unit_list = nn.ModuleList([(LSTMunit(self.dummy_list[i], self.dummy_list[i+1], kernel_size).double()).cuda() for i in range(len(self.test_input))])
        
        if self.debug:
            print("number of units:")
            print(len(self.unit_list))
#             print("number of ")

#         self.unit_list = nn.ModuleList(self.unit_list)
    
    
    def forward(self, x, copy_in = False, copy_out = [False, False, False]):
#     def forward(self, x):
#         copy_in = False
#         copy_out = [False, False, False]

        
#         print("IS X CUDA?")
#         print(x.is_cuda)
        """loop over layers, then over hidden states
        
        copy_in is either False or is [[h,c],[h,c]] ect.
        
        THIS IN NOW CHANGED TO COPY IN 
        
        """
        
        internal_outputs = []
        """TODO: HOW MANY OUTPUTS TO SAVE"""
        """ S """
        
        """ TODO: PUT INITIAL ZERO THROUGH THE SYSTEM TO DEFINE H AND C"""
        
        layer_output = [] # empty list to save each h and c for each step. 
        """TODO: DECIDE WHETHER THE ABOVE SHOULD BE ARRAY OR NOT"""
        
        # x is 5th dimensional tensor.
        # x is of size batch, sequence, layers, height, width
        
        """TODO: INITIALISE THESE WITH VECTORS."""
        # these need to be of dimensions (batchsizze, hidden_dim, heigh, width)
        
        size = x.shape
        
        # need to re arrange the outputs. 
        
        
        """TODO: SORT OUT H SIZING. """
        
        batch_size = size[0]
        # change this. h should be of dimensions hidden size, hidden size.
        h_shape = list(x.shape[:1] + x.shape[2:]) # seq is second, we miss it with fancy indexing
        h_shape[1] = self.hidden_chans
        if self.debug:
            print("h_shape:")
            print(h_shape)
        
        # size should be (seq, batch_size, layers, height, weight)
        
        
        empty_start_vectors = []
        
        
        for i in range(self.layers):
            """CHANGED: NOW HAS COPY IN COPY OUT BASED ON [[0,0][H,C]] FORMAT"""
            if copy_in == False: # i.e if no copying in occurs then proceed as normal
                h_shape = list(x.shape[:1] + x.shape[2:]) # seq is second, we miss it with fancy indexing
                h_shape[1] = self.dummy_list[i+1] # check indexing. 
#                 empty_start_vectors.append([(torch.zeros(h_shape).double()).cuda(), (torch.zeros(h_shape).double()).cuda()])
                empty_start_vectors.append([(torch.zeros(h_shape).double()).cuda(), (torch.zeros(h_shape).double()).cuda()])
#             elif copy_in[i] == [0,0]:
            elif isinstance(copy_in[i], list):

                assert (len(copy_in) == self.layers), "Length disparity between layers, copy in format"

                # if no copying in in alternate format
                h_shape = list(x.shape[:1] + x.shape[2:]) # seq is second, we miss it with fancy indexing
                h_shape[1] = self.dummy_list[i+1] # check indexing. 
                empty_start_vectors.append([(torch.zeros(h_shape).double()).cuda(), (torch.zeros(h_shape).double()).cuda()])
                
            else: # copy in the provided vectors
                assert (len(copy_in) == self.layers), "Length disparity between layers, copy in format"

                """TODO: DECIDE WHETHER TO CHANGE THIS TO AN ASSERT BASED OFF TYPE OF TENSOR."""
                empty_start_vectors.append(copy_in[i])
                
            
        
        
        
#         empty_start_vectors = [[torch.zeros(h_shape), torch.zeros(h_shape)] for i in range(self.layers)]
        
        
        
        if self.debug:
            for i in empty_start_vectors:
                print(i[0].shape)
            print(" \n \n \n")
        
#         for i in range(self.layers):
#             empty_start_vectors.append([torch.tensor()])
        
        total_outputs = []
        
        
        for i in range(self.layers):
            
            
            layer_output = []
            if self.debug:
                print("layer iteration:")
                print(i)
            # for each in layer

            """AS WE PUT IN ZEROS EACH TIME THIS MAKES OUR LSTM STATELESS"""
            # initialise with zero or noisy vectors 
            # at start of each layer put noisy vector in 
            # look at tricks paper to find more effective ideas of how to put this in
            # do we have to initialise with 0 tensors after we go to the second layer
            # or does the h carry over???
            """TODO: REVIEW THIS CHANGE"""
            
            # copy in for each layer. 
            # this is used for encoder decoder architectures.
            # default is to put in empty vectors. 
            
            """TODO: REVIEW THIS SECTION"""
            """CHANGED: TO ALWAYS CHOOSE H AND C"""
#             if copy_in == False:
#                 h, c = empty_start_vectors[i]
#             else: h, c = copy_in[i]

            h, c = empty_start_vectors[i] 
                
            if self.debug:
                print("new h shape")
                print(h.shape)
                
            """TODO: DO WE HAVE TO PUT BLANK VECTORS IN AT EACH TIMESTEP?"""
            
            # need to initialise zero states for c and h. 
            for j in range(self.seq_length):
                if self.debug:
                    print("inner loop iteration:")
                    print(j)
                if self.debug:
                    print("x dtype is:" , x.dtype)
                # for each step in the sequence
                # put x through 
                # i.e put through each x value at a given time.
                
                """TODO: PUT H IN FROM PREVIOUS LAYER, BUT C SHOULD BE ZEROS AT START"""
                
                if self.debug:
                    print("inner loop size:")
                    print(x[:,j].shape)
                    print("h size:")
                    print(h.shape)
                    
                h, c = self.unit_list[i](x[:,j], h, c)
                
                # this is record for each output in given layer.
                # this depends whether copying out it enabld 
#                 i
                layer_output.append([h, c])
                
            """TODO: IMPLEMENT THIS"""
#             if self.save_all_outputs[i]:
#                 total_outputs.append(layer_outputs[:,0]) # saves h from each of the layer outputs
                
            # output 
            if copy_out[i] == True:
                # if we want to copy out the contents of this layer:
                internal_outputs.append(layer_output[-1])
                # saves last state and memory which can be subsequently unrolled.
                # when used in an encoder decoder format.
            
            else:
                internal_outputs.append([0,0])
                # saves null variable so we can check whats being sent out.
            
            
            h_output = [i[0] for i in layer_output] #layer_output[:,0] # take h from each timestep.
            if self.debug:
                print("h_output is of size:")
                print(h_output[0].shape)
                
                      
            """TODO: REVIEW IF 1 IS THE CORRECT AXIS TO CONCATENATE THE VECTORS ALONG"""
            # we now use h as the predictor input to the other layers.
            """TODO: STACK TENSORS ALONG NEW AXIS. """
            
            
            x = torch.stack(h_output,0)
            x = torch.transpose(x, 0, 1)
            if self.debug:
                print("x reshaped dimensions:")
                print(x.shape)
        
#         x = torch.zeros(x.shape)
#         x.requires_grad = True
        return x , internal_outputs # return new h in tensor form. do we need to cudify this stuff

    def initialise(self):
        """put through zeros to start everything"""
        

In [0]:
# bytes = torch.cuda.memory_allocated()
# print("amount of memory allocated: ", bytes / 1073741824)

In [18]:
shape = [2,4,1,8,8] # batch size, seq length, 1 layer, 32 by 32 image.
# import numpy as np
# x = (torch.randn(shape).double()).cuda()

test2 = LSTMmain(shape, 1, 3, 5, [1], test_input = [1,2], debug = False).double()


printm()
# test2 = (LSTMmain(shape, 1, 3, 5, [1], test_input = [1], debug = False)).to(device)
# test2.cuda()
# # x.cuda()
# print("IS X CUDA NOW?")
# print(x.is_cuda)


# print("x_shape:")

# print(x.shape)

# ans, _ = test2(x, copy_in = False, copy_out = [False, False, False])

# ans.shape
# ans = ans.double()
# t1 = np.zeros(shape)
# t1 = torch.tensor(t1, requires_grad = True).to(device)
# t1 = t1.cuda()
# t1 = torch.FloatTensor([2,4,1,8,8], dtype = torch.float, requires_grad = True)
# print(t1.requires_grad)
# # print(ans.requires_grad)
# # res = torch.autograd.gradcheck(test2, (t1,), eps=1e-4, raise_exception=True)
# print(res)
# print(res)
# torch.autograd.gradcheck(test2, (ans,))
# a = list(x.shape)
# print(a[:0] + a[1:])

# a = torch.randn([20,19,32,32])
# for i in range(19):

#     print(a[:,i])
# a = test2(x)
# b = test2(x)



# a - b

Gen RAM Free: 11.3 GB  | Proc size: 2.2 GB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [0]:
# res

In [0]:
# for param in test2.parameters():
#     print(type(param.data), param.size(), param.is_cuda, param.name)
    

In [0]:
# print(test2)

In [0]:
# for param in test2.parameters():
#     print(param.device)

In [0]:
# res

In [0]:
# summary(test2, input_size = (1,4,1,8,8))

# ENCODER DECODER MODEL

In [0]:
test2 = LSTMmain(shape, 1, 3, 5, [1], test_input = [1,2], debug = False).double()

class LSTMencdec(nn.Module):
    """structure is overall architecture of """
    def __init__(self, structure, input_channels, kernel_size = 5, debug = True):
        super(LSTMencdec, self).__init__()
#         assert isinstance(structure, np.array), "structure should be a 2d numpy array"
        assert len(structure.shape) == 2, "structure should be a 2d numpy array with two rows"
        self.debug = debug
        
        """TODO: MAKE KERNEL SIZE A LIST SO CAN SPECIFY AT EACH JUNCTURE."""
        shape = [1,10,1,64,64]
        
        self.structure = structure
        """STRUCTURE IS AN ARRAY - CANNOT USE [] + [] LIST CONCATENATION - WAS ADDING ONE ONTO THE ARRAY THING."""
        self.input_channels = input_channels
        self.kernel_size = kernel_size
        
        """TODO: ASSERT THAT DATATYPE IS INT."""
        
        self.enc_shape, self.dec_shape, self.enc_copy_out, self.dec_copy_in = self.input_test()
        
        if self.debug:
            print("enc_shape, dec_shape, enc_copy_out, dec_copy_in:")
            print(self.enc_shape)
            print(self.dec_shape)
            print(self.enc_copy_out)
            print(self.dec_copy_in)
            
        
        
        self.encoder = LSTMmain(shape, self.input_channels, len(self.enc_shape)+1, self.kernel_size, layer_output = self.enc_copy_out, test_input = self.enc_shape).cuda()
        
        self.decoder = LSTMmain(shape, self.enc_shape[-1], len(self.dec_shape), self.kernel_size, layer_output = 1, test_input = self.dec_shape).cuda()
        
        
        
        # initialise encoder and decoder network
    
    def input_test(self):
        """check input structure to make sure there is overlap between encoder 
        and decoder.
        """
        copy_grid = []
        # finds dimensions of the encoder
        enc_layer = self.structure[0]
        enc_shape = enc_layer[enc_layer!=0]
        dec_layer = self.structure[1]
        dec_shape = dec_layer[dec_layer!=0]
#         
        
        
        
        
        
        #set up boolean grid of where the overlaps are.
        for i in range(len(enc_layer)):
            if self.debug:
                print(enc_layer[i], dec_layer[i])
            if (enc_layer[i] != 0) and (dec_layer[i] != 0):
                copy_grid.append(True)
            else:
                copy_grid.append(False)
                
                
        enc_overlap = copy_grid[:len(enc_layer)-1]
        
        num_dec_zeros = len(dec_layer[dec_layer==0]) # will this break if no zeros?
        
        dec_overlap = copy_grid[num_dec_zeros:]
        
        return enc_shape, dec_shape, enc_overlap, dec_overlap
        
#         dec_overlap = copy_grid[]                
        
                
                
#         [[1,2,3,0],
#          [0,2,3,1]]
    def forward(self, x):
        
        x, out_states = self.encoder(x, copy_in = False, copy_out = self.enc_copy_out)
        
        dummy_input = torch.zeros(x.shape)
        
        res, _ = self.decoder(x, copy_in = out_states, copy_out = [False, False, False])
        print("FINISHING ONE PASS")
        return res

In [0]:
# structure = np.array([[2,2,2,0],[0,2,2,1]])
# shape = [2,4,1,8,8]

# x = torch.randn(shape).double()


# print(structure.shape)
# # this we will also use an input channel no of 1. 
# # we then anticipate channels 2, 2 -> decoder : 2, 1 -> output. last channel of decoder is 1 as we then need to narrow down 
# # to parameters of the size we need for output
# test = LSTMencdec(structure, 1)
# test(x)

In [0]:
# import numpy as np
# a = np.array([[1,2,3,4],[4,5,6,7]])

In [0]:
# # b = a[0]
# b[b!=1]

# Dataset and Data Loader

## DataSet

In [0]:
# train_set.
# test_set = MovingMNIST(root='.data/mnist', train=False, download=True)
# is of shape list(1000), tuple - start and finish. 




In [0]:
# test_set[1][1].shape
# input is batch_size, tuple(prev, after), seq(10,), height, width
# need to go to batch, seq, chan, height, width.

# print(test_set[:][0].shape) # of size 1000, 10, 64, 64

In [0]:


# # test_set[0][0].shape
# class SequenceDataset(Dataset):
#     """simple data set wrapper 
#     for the moving mnist dataset
    
#     we use this as need to insert channel dimension in the data"""
#     def __init__(self, data, transform = None):
        
#         self.input_sequence = data[:][0].unsqueeze(2)
        
#         self.output_sequence = data[:][1].unsqueeze(2)# this should be the moving mnist sent in
        
#         assert len(self.input_sequence.shape) == 5, "internal data should be : batch_size, sequence_length, channels, height, width"
        
        
# #         print(self.input_sequence.shape)
        
        
#     def __len__(self):
#         return len(self.input_sequence)
    
#     def __getitem__(self, i):
#         """returns tuple of predictor and result sequence
        
#         This should later be specified to return a valid number of steps in the future
        
#         i.e can specify whether want input of 10 and to predict 5 ect."""
                

        
#         return self.input_sequence[i], self.output_sequence[i]
    
        

In [0]:
class SequenceDataset(Dataset):
    """simple data set wrapper 
    for the moving mnist dataset
    
    we use this as need to insert channel dimension in the data"""
    def __init__(self, predictor, ground_truth, transform = None):
        
        self.input_sequence = predictor
        
        self.output_sequence = ground_truth
        
        assert len(self.input_sequence.shape) == 5, "internal data should be : batch_size, sequence_length, channels, height, width"
        
        
#         print(self.input_sequence.shape)
        
       
    def __len__(self):
        return len(self.input_sequence)
    
    def __getitem__(self, i):
        """returns tuple of predictor and result sequence
        
        This should later be specified to return a valid number of steps in the future
        
        i.e can specify whether want input of 10 and to predict 5 ect."""
                

        
        return self.input_sequence[i], self.output_sequence[i]
    

## HDF5 DATASET AND INITIALISE

In [0]:
class HDF5Dataset(Dataset):
    """dataset wrapper for hdf5 dataset to allow for lazy loading of data. This 
    allows ram to be conserved. 
    
    As the hdf5 dataset is not partitioned into test and validation, the dataset 
    takes a shuffled list of indices to allow specification of training and 
    validation sets.
    
    MAKE SURE TO CALL DEL ON GENERATED OBJECTS OTHERWISE WE WILL CLOG UP RAM
    
    """
    
    def __init__(self, path, index_map, transform = None):
        
        %cd /content/drive/My \Drive/masters_project/data 
        # changes directory to the one where needed.
        
        self.path = path
        
        self.index_map = index_map # maps to the index in the validation split
        # due to hdf5 lazy loading index map must be in ascending order.
        # this may be an issue as we should shuffle our dataset.
        # this will be raised as an issue as we consider a work around.
        # we should keep index map shuffled, and take the selection from the 
        # shuffled map and select in ascending order. 
        
        
        self.file = h5py.File(path, 'r')
        
        
        
        
        
        
    def __len__(self):
        return len(self.index_map)
    
    def __getitem__(self,i):
        
        i = self.index_map[i] # index maps from validation set to select new orders
#         print(i)
        if isinstance(i, list): # if i is a list. 
            i.sort() # sorts into ascending order as specified above
            
        """TODO: CHECK IF THIS RETURNS DOUBLE"""
        
        predictor = torch.tensor(self.file["predictor"][i])
        
        truth = torch.tensor(self.file["truth"][i])
        
        return predictor, truth

In [0]:
def initialise_dataset_HDF5(valid_frac = 0.1, dataset_length = 9000):
    """
    Returns datasets for training and validation. 
    
    Loads in datasets segmenting for validation fractions.
   
    
    
    """
    
    if valid_frac != 0:
        
        dummy = np.array(range(dataset_length)) # clean this up - not really needed
        
        train_index, valid_index = validation_split(dummy, n_splits = 1, valid_fraction = 0.1, random_state = 0)
        
        train_dataset = HDF5Dataset("train_set.hdf5", index_map = train_index)
        
        valid_dataset = HDF5Dataset("test_set.hdf5", index_map = valid_index)
        
        return train_dataset, valid_dataset
        
    else:
        print("not a valid fraction for validation") # turn this into an assert.
        
        
        

In [0]:
# test[0].shape

## Data Loader
num_workers parameter is useful for bypassing large data set issues. this is very relevant for future work.



#TRAINING FUNCTIONS - ENC DEC


## Load in datasets

In [0]:
# train_set = MovingMNIST(root='.data/mnist', train=True, download=True)
# test_set = MovingMNIST(root='.data/mnist', train=False, download=True)

In [34]:
data = 10
dummy_array = np.zeros(data)
split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.1, random_state = 32)
generator = split.split(torch.tensor(dummy_array), torch.tensor(dummy_array))
indices = [(a, b) for a, b in generator][0]
print(indices)

# for a, b in generator:
#     print(a, b)

(array([9, 4, 6, 5, 3, 1, 2, 0, 8]), array([7]))


## Function to shuffle the dataset 

layter include kfold validaiton. 

In [0]:
def validation_split(data, n_splits = 1, valid_fraction = 0.1, random_state = 0):
    """
    Function to produce a validation set from test set.
    THIS SHUFFLES THE SAMPLES. __NOT__ THE SEQUENCES.
    """
    dummy_array = np.zeros(len(data))
    split = StratifiedShuffleSplit(n_splits, test_size = valid_fraction, random_state = 0)
    generator = split.split(torch.tensor(dummy_array), torch.tensor(dummy_array))
    return [(a,b) for a, b in generator][0]


## Unsqueeze data

In [0]:
def unsqueeze_data(data):
    """
    Takes in moving MNIST object - must then account for 
    """
    
    # split moving mnist data into predictor and ground truth.
    predictor = data[:][0].unsqueeze(2)
    predictor = predictor.double()
        
    truth = data[:][1].unsqueeze(2)# this should be the moving mnist sent in
    truth = truth.double()
    
    return predictor, truth
    # the data should now be unsqueezed.


## Produce Datasets

In [0]:
def initialise_dataset(data):
    # unsqueeze data, adding a channel dimension for later convolution. 
    # this also gets rid of the annoying tuple format
    predictor, truth = unsqueeze_data(data)
    
    train_index, valid_index = validation_split(data)
    
    train_predictor = predictor[train_index]
    valid_predictor = predictor[valid_index]
    
    train_truth = truth[train_index]
    valid_truth = truth[valid_index]
    
    train_dataset = SequenceDataset(train_predictor, train_truth)
    valid_dataset = SequenceDataset(valid_predictor, valid_truth)
    
    return train_dataset, valid_dataset

### Test of produce dataset function

In [38]:
## test 
printm()
train, valid = initialise_dataset_HDF5()

printm()





Gen RAM Free: 11.3 GB  | Proc size: 2.2 GB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB
/content/drive/My Drive/masters_project/data
/content/drive/My Drive/masters_project/data
Gen RAM Free: 11.3 GB  | Proc size: 2.2 GB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [49]:
# plt.imshow(train[0][0][0][0])
train[0][0][0][0][30][30]

tensor(254., dtype=torch.float64)

As we can see below, works as intended. note that shape now has a channel , which must be specified.

In [0]:
# for i in range(10):
#     plt.figure()
#     plt.imshow(train[0][0][i][0].numpy())

# for i in range(10):
#     plt.figure()
#     plt.imshow(train[0][1][i][0].numpy())

## patch size alteration

## Define Training Functions 

In [0]:

def train_enc_dec(model, optimizer, dataloader, loss_func = nn.MSELoss()):
    """
    training function 
    
    by default mseloss
    
    could try brier score.
    
    """
    i = 0
    model.train() # enables training for model. 
    tot_loss = 0
    for x, y in dataloader:
#         print("training")
        x = x.to(device) # send to cuda.
        y = y.to(device)
        optimizer.zero_grad() # zeros saved gradients in the optimizer.
        # prevents multiple stacking of gradients
        # this is important to do before we evaluate the model as the 
        # model is currenly in model.train() mode
        
        prediction = model(x) #x should be properly formatted - of size
        """THIS DOESNT DEAL WITH SEQUENCE LENGTH VARIANCE OF PREDICTION OR Y"""
        loss = loss_func(prediction, y)
        
        loss.backward() # differentiates to find minimum.
        
        ##
        # implement the interpreteable stuff here.
        # as it is very unlikely we predict every pixel correctly we will not 
        # use accuracy. 
        # technically this is a regression problem, not a classification.
        
        
        optimizer.step() # steps forward the optimizer.
        # uses loss.backward() to give gradient. 
        # loss is negative.
#         del x # make sure the garbage is collected.
#         del y
        tot_loss += loss 
        print("BATCH:")
        print(i)
        i += 1
#         if i == 20:
#             break
        print("MSE_LOSS:", tot_loss / i)
    return model # trainloss, trainaccuracy 

def validate(model, dataloader, criterion = nn.MSELoss()):
    """as for train_enc_dec but without training - and acting upon validation
    data set
    """
    model.eval() # puts out of train mode so we do not mess up our gradients
    for x, y in dataloader:
        with torch.no_grad: # no longer have to specify tensors 
            # as volatile = True. as of modern pytorch use torch.no_grad.
            
            x.to(device) # send to cuda.
            y.to(device)
            prediction = model(x)
            
            loss = loss_func(prediction, y)
            
            
    return validloss, validaccuracy





def train_main(model, params, train, valid, epochs = 30, batch_size = 1):
    # make sure model is ported to cuda
    # make sure seed has been specified if testing comparative approaches
    
#     if model.is_cuda == False:
#         model.to(device)
    
    # initialise optimizer on model parameters 
    # chann
    optimizer = torch.optim.Adam(model.parameters())
    loss_func = nn.MSELoss()
    
    train_loader = DataLoader(train, batch_size = batch_size, shuffle = True) # implement moving MNIST data input
    validation_loader = DataLoader(valid, batch_size = batch_size, shuffle = False) # implement moving MNIST
    
    for epoch in range(epochs):
        
        train_enc_dec(model, optimizer, train_loader)
        
        
        torch.save(optimizer.state_dict(), F"Adam_train"+str(epoch)+".pth")
        torch.save(model.state_dict(), F"Test_train_model"+str(epoch)+".pth")
#         validate(model, validation_loader)
        
    return model, optimizer
        
    
    
    
            
            
            
            
    


        
        
        
        
        
        
        
        
        
        
        
        

# TEST OF FULL ENC DEC


In [0]:
# train, valid = initialise_dataset(train_set)
# print(train[0][0].dtype)

# dat = DataLoader(train, batch_size = 20)

In [43]:
%pwd

'/content/drive/My Drive/masters_project/data'

In [44]:
printm()

Gen RAM Free: 11.3 GB  | Proc size: 2.2 GB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [45]:

structure = np.array([[4,8,0],[0,8,1]])

test_model = LSTMencdec(structure, 1).to(device)

# optim = torch.optim.Adam(test_model.parameters())



# train_enc = train_enc_dec(test_model,)


# train_main(test_model, 1, train, valid, epochs = 2, batch_size = 50)

model, optimizer = train_main(test_model, 1, train, valid, epochs = 20, batch_size = 50)


torch.save(optimizer.state_dict(), F"Finished_opt.pth")
torch.save(model.state_dict(), F"Finished_mod.pth")





4 0
8 8
0 1
enc_shape, dec_shape, enc_copy_out, dec_copy_in:
[4 8]
[8 1]
[False, True]
[True, False]
FINISHING ONE PASS
BATCH:
0
MSE_LOSS: tensor(2731.4474, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
FINISHING ONE PASS
BATCH:
1
MSE_LOSS: tensor(2715.5688, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
FINISHING ONE PASS
BATCH:
2
MSE_LOSS: tensor(2727.9422, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
FINISHING ONE PASS
BATCH:
3
MSE_LOSS: tensor(2710.6953, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
FINISHING ONE PASS
BATCH:
4
MSE_LOSS: tensor(2709.2750, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
FINISHING ONE PASS
BATCH:
5
MSE_LOSS: tensor(2731.4207, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
FINISHING ONE PASS
BATCH:
6
MSE_LOSS: tensor(2726.3968, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
FINISHING ONE PASS
BATCH:
7
MSE_LOSS: tensor(2741.5847, device='cuda:0', 

KeyboardInterrupt: ignored

In [0]:
test_model.to(device)
for params in test_model.parameters():
    print(params.is_cuda)
    print(params.data)

# MODEL PRINTING FOR ENC DEC


In [0]:
# x = torch.randn([1,10,1,64,64]).double()
# y = test_model(x)

In [0]:
# from torchviz import make_dot

In [0]:
# graph = make_dot(y, params = dict(test_model.named_parameters()))


In [0]:
%pwd

In [0]:
# graph.render(format = 'pdf')

In [0]:
# graph


In [0]:
# train_main(test_model, 1, train, valid, epochs = 2, batch_size = 5)


C:\Users\Gareth\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:443: UserWarning: Using a target size (torch.Size([1, 10, 1, 64, 64])) that is different to the input size (torch.Size([1, 10, 2, 64, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction) 
  
error message from above - why is there an increase in channle size / number?? 
